In [6]:
import pandas as pd
import numpy as np


import os # for file system operations
import re # for regular expressions
from tqdm import tqdm # for progress bar

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
list_xlsx_files = os.listdir('../data/input/population')
list_xlsx_files

['SB_A01-16-00_2018h02_BE.xlsx',
 'SB_A01-16-00_2019h02_BE.xlsx',
 'SB_A01-16-00_2020h02_BE.xlsx',
 'SB_A01-16-00_2021h02_BE.xlsx',
 'SB_A01-16-00_2022h02_BE.xlsx',
 'SB_A01-16-00_2023h02_BE.xlsx']

In [8]:
def transform_number_under_10(x):
    if x < 10:
        x = '0' + str(x)
        return x
    else:
        x = str(x)
        return x

In [9]:
column_names = [i for i in range(0, 15)]

population_df = pd.DataFrame(columns=column_names)
display(population_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14


In [10]:
# Loop through all the files in the directory
for filename in tqdm(list_xlsx_files): 

    print(f'{filename}')
    
    # extract year and half_year from filename
    temp = re.search(r'(\d{4})h(\d{2})', filename)
    year, half_year = temp.groups()

    # create file path for file import 
    file_path = f'../data/input/population/{filename}'

    # import data from 'T2' sheet - checked in Excel 
    raw_data = pd.read_excel(file_path, 'T2', header = None)

    # checked with excel the dataset before -> after dropping rows with Nan values the needed information are extracted 
    final_rows = raw_data.dropna().index.tolist()
    pop_data = raw_data.iloc[final_rows]
        
    # Add 2 columns year and half_year
    pop_data.insert(0, 'year', year)
    pop_data.insert(1, 'half_year', half_year)
    
    # counter = counter + len(pop_data)
        
    population_df = pd.concat([population_df, pop_data], ignore_index=True , axis=0)

    # Convert columns to integer 
    columns_to_transform = population_df.columns[0:17]
    population_df[columns_to_transform] = population_df[columns_to_transform].astype(int)
    # print(population_df.dtypes)
    
    # Transform format for LOR values - (1 -> 01, 2 -> 02 ...)
    for col in [0,1,2,3]:
        population_df[col] = population_df[col].apply(lambda x: transform_number_under_10(x))
    


    # drop woman and asyl data columns 
    # transform 0-3 to LOR Code
    # Rearrange table columns    
    population_df['lor'] = population_df[0] + population_df[1] + population_df[2] + population_df[3]
    population_df['key'] = population_df['lor'] + '-' + population_df['year'].astype(str) + '-' +  population_df['half_year'].astype(str)
    
# Drop unnecessary columns 
population_df = population_df.drop(columns = [0, 1, 2, 3, 13, 14])

# Reorder and rename columns 
population_df = population_df.reindex(['key', 'year', 'half_year', 'lor', 4, 5, 6, 7, 8, 9, 10, 11, 12], axis = 1)
population_df.columns = ['key', 'year', 'half_year', 'lor', 'total_population', '-6', '6-15', '15-18', '18-27', '27-45', '45-55', '55-65', '65+']


# # Save the final DataFrame to a pickle file 
# population_df.to_pickle('../data/output/temp_analysis/total_population_dataset.pkl')
# # population_df.to_excel('../data/output/total_population_dataset.xlsx', index = False)
# # population_df.to_csv('../data/output/total_population_dataset.csv', index = False)

display(population_df)

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

SB_A01-16-00_2018h02_BE.xlsx


C:\Users\Domen\AppData\Local\Temp\ipykernel_3324\3045370019.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  population_df = pd.concat([population_df, pop_data], ignore_index=True , axis=0)
 17%|██████████████                                                                      | 1/6 [00:00<00:02,  1.70it/s]

SB_A01-16-00_2019h02_BE.xlsx


 33%|████████████████████████████                                                        | 2/6 [00:00<00:01,  2.77it/s]

...

SB_A01-16-00_2022h02_BE.xlsx


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.08it/s]

SB_A01-16-00_2023h02_BE.xlsx


100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  3.51it/s]


,key,year,half_year,lor,total_population,-6,6-15,15-18,18-27,27-45,45-55,55-65,65+
0,01011101-2018-2,2018,2,01011101,3384,178,120,38,493,1086,359,383,727
1,01011102-2018-2,2018,2,01011102,196,3,3,3,30,58,39,38,22
2,01011103-2018-2,2018,2,01011103,5038,243,325,109,589,1459,645,742,926
3,01011104-2018-2,2018,2,01011104,4759,261,394,143,851,1689,601,457,363
4,01011105-2018-2,2018,2,01011105,1421,83,87,15,132,475,226,180,223
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964,12601032-2023-2,2023,2,12601032,6115,493,814,266,698,1477,692,653,1022
2965,12601133-2023-2,2023,2,12601133,11441,742,1405,472,1283,2762,1389,1272,2116
2966,12601134-2023-2,2023,2,12601134,15947,1174,1854,577,1792,3601,1771,1848,3330
2967,12601235-2023-2,2023,2,12601235,10625,758,1412,468,1300,2503,1240,1210,1734


In [11]:
population_df.to_csv('../data/tableau/population.csv', index = False)